# Gallery Example: Hyper/Erl/1 Feedback Queue

This example demonstrates a HyperExp/Erlang/1 queueing system with feedback routing:
- **Arrivals**: Hyperexponential distribution (high variability)
- **Service**: Erlang distribution (low variability)
- **Servers**: 1 server
- **Scheduling**: FCFS
- **Feedback**: Jobs may return to the queue with some probability

In feedback queues, jobs may loop back and require additional service.

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_hyperl1_feedback():
    """Create Hyper/Erl/1-Feedback queueing model"""
    model = Network('Hyper/Erl/1-Feedback')
    
    # Block 1: nodes
    source = Source(model, 'Source')
    queue = Queue(model, 'Queue', SchedStrategy.FCFS)
    sink = Sink(model, 'Sink')
    
    # Block 2: classes
    oclass1 = OpenClass(model, 'Class1')
    
    # HyperExp arrival (SCV=64)
    source.set_arrival(oclass1, HyperExp.fit_mean_and_scv(1, 64))
    
    # Erlang service
    queue.set_service(oclass1, Erlang.fit_mean_and_order(0.5, 5))
    
    # Block 3: topology with feedback (90% feedback probability)
    P = model.init_routing_matrix()
    P.add_route(oclass1, source, queue, 1.0)
    P.add_route(oclass1, queue, queue, 0.9)  # 90% loop back
    P.add_route(oclass1, queue, sink, 0.1)   # 10% exit
    model.link(P)
    
    return model

# Create the model
model = gallery_hyperl1_feedback()

## About Feedback Queues

In feedback queues:
- Jobs may require multiple passes through the server
- After each service, there's a probability of returning
- Mean number of visits = 1 / (1 - feedback_probability)

With 90% feedback probability:
- Mean visits per job = 1 / 0.1 = 10
- Effective arrival rate at queue = external_rate × mean_visits
- Must ensure stability: effective_rate < service_rate

This creates high utilization and potential congestion.

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)